In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import KFold

class Thresholder:
    """
    اعمال آستانه‌گذاری روی ستون احتمالات و افزودن نتیجه به دیتاست.

    Parameters
    ----------
    method : str
        نام روش آستانه‌گذاری: 'FixedThreshold', 'OptimalThreshold', 'AdaptiveThreshold'
    config : dict
        تنظیمات مخصوص هر روش.

        برای FixedThreshold:
            - threshold : float, default=0.5
            - output_column : str, default='predicted_class'
            - reverse : bool, default=False (اگر True، مقادیر کمتر از آستانه کلاس مثبت در نظر گرفته شوند)

        برای OptimalThreshold:
            - target_column : str, required (نام ستون هدف واقعی)
            - metric : str, default='f1' (معیار بهینه‌سازی: 'f1', 'accuracy', 'precision', 'recall', 'youden')
            - cv : int, default=5 (تعداد فولدها برای اعتبارسنجی، 1 یعنی از کل داده استفاده کن)
            - step : float, default=0.01 (گام جستجوی آستانه)
            - maximize : bool, default=True (اگر False، معیار کمینه شود)
            - output_column : str, default='predicted_class'

        برای AdaptiveThreshold:
            - method : str, default='quantile' (روش تطبیقی: 'quantile', 'knn')
            - window : int, default=10 (اندازه پنجره برای quantile)
            - quantile : float, default=0.5 (چندک برای quantile)
            - n_neighbors : int, default=5 (تعداد همسایه‌ها برای knn)
            - distance_metric : str, default='euclidean'
            - output_column : str, default='predicted_class'
    """
    def __init__(self, method, config):
        self.method = method
        self.config = config
        self.fitted_threshold_ = None  # برای OptimalThreshold

    def transform(self, dataset, column):
        """
        اعمال آستانه‌گذاری روی ستون مشخص و افزودن نتیجه.

        Parameters
        ----------
        dataset : pd.DataFrame
            دیتاست ورودی.
        column : str
            نام ستون حاوی احتمالات (یا نمرات).

        Returns
        -------
        pd.DataFrame
            دیتاست با ستون جدید (نام ستون خروجی در config مشخص شده یا پیش‌فرض 'predicted_class').
        """
        if not isinstance(dataset, pd.DataFrame):
            raise TypeError("dataset must be a pandas DataFrame")
        if column not in dataset.columns:
            raise ValueError(f"Column '{column}' not found in dataset")

        dataset = dataset.copy()
        output_col = self.config.get('output_column', 'predicted_class')

        if self.method == 'FixedThreshold':
            return self._fixed_threshold(dataset, column, output_col)
        elif self.method == 'OptimalThreshold':
            return self._optimal_threshold(dataset, column, output_col)
        elif self.method == 'AdaptiveThreshold':
            return self._adaptive_threshold(dataset, column, output_col)
        else:
            raise ValueError(f"Unknown threshold method: {self.method}")

    def _fixed_threshold(self, dataset, column, output_col):
        """آستانه ثابت."""
        threshold = self.config.get('threshold', 0.5)
        reverse = self.config.get('reverse', False)
        probs = dataset[column].values
        if reverse:
            preds = (probs < threshold).astype(int)
        else:
            preds = (probs >= threshold).astype(int)
        dataset[output_col] = preds
        return dataset

    def _optimal_threshold(self, dataset, column, output_col):
        """یافتن آستانه بهینه بر روی داده و اعمال آن."""
        target_col = self.config.get('target_column')
        if target_col is None:
            raise ValueError("OptimalThreshold requires 'target_column' in config.")
        if target_col not in dataset.columns:
            raise ValueError(f"Target column '{target_col}' not found in dataset.")

        y_true = dataset[target_col].values
        y_score = dataset[column].values
        metric = self.config.get('metric', 'f1')
        cv = self.config.get('cv', 5)
        step = self.config.get('step', 0.01)
        maximize = self.config.get('maximize', True)

        if cv > 1:
            # اعتبارسنجی متقاطع برای انتخاب آستانه
            kf = KFold(n_splits=cv, shuffle=True, random_state=42)
            thresholds = np.arange(0.0, 1.0 + step, step)
            scores = []
            for train_idx, val_idx in kf.split(y_score):
                y_true_train, y_true_val = y_true[train_idx], y_true[val_idx]
                y_score_train, y_score_val = y_score[train_idx], y_score[val_idx]

                best_thresh, best_score = None, -np.inf if maximize else np.inf
                for thresh in thresholds:
                    preds = (y_score_val >= thresh).astype(int)
                    if metric == 'f1':
                        score = f1_score(y_true_val, preds, zero_division=0)
                    elif metric == 'accuracy':
                        score = accuracy_score(y_true_val, preds)
                    elif metric == 'precision':
                        score = precision_score(y_true_val, preds, zero_division=0)
                    elif metric == 'recall':
                        score = recall_score(y_true_val, preds, zero_division=0)
                    elif metric == 'youden':
                        tp = ((preds == 1) & (y_true_val == 1)).sum()
                        tn = ((preds == 0) & (y_true_val == 0)).sum()
                        fn = ((preds == 0) & (y_true_val == 1)).sum()
                        fp = ((preds == 1) & (y_true_val == 0)).sum()
                        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
                        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
                        score = sensitivity + specificity - 1
                    else:
                        raise ValueError(f"Unsupported metric: {metric}")

                    if (maximize and score > best_score) or (not maximize and score < best_score):
                        best_score = score
                        best_thresh = thresh
                scores.append(best_thresh)

            optimal_threshold = np.mean(scores)
        else:
            # از کل داده برای انتخاب آستانه استفاده کن (بدون اعتبارسنجی)
            best_thresh, best_score = None, -np.inf if maximize else np.inf
            for thresh in np.arange(0.0, 1.0 + step, step):
                preds = (y_score >= thresh).astype(int)
                if metric == 'f1':
                    score = f1_score(y_true, preds, zero_division=0)
                elif metric == 'accuracy':
                    score = accuracy_score(y_true, preds)
                elif metric == 'precision':
                    score = precision_score(y_true, preds, zero_division=0)
                elif metric == 'recall':
                    score = recall_score(y_true, preds, zero_division=0)
                elif metric == 'youden':
                    tp = ((preds == 1) & (y_true == 1)).sum()
                    tn = ((preds == 0) & (y_true == 0)).sum()
                    fn = ((preds == 0) & (y_true == 1)).sum()
                    fp = ((preds == 1) & (y_true == 0)).sum()
                    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
                    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
                    score = sensitivity + specificity - 1
                else:
                    raise ValueError(f"Unsupported metric: {metric}")

                if (maximize and score > best_score) or (not maximize and score < best_score):
                    best_score = score
                    best_thresh = thresh
            optimal_threshold = best_thresh

        self.fitted_threshold_ = optimal_threshold
        reverse = self.config.get('reverse', False)
        probs = dataset[column].values
        if reverse:
            preds = (probs < optimal_threshold).astype(int)
        else:
            preds = (probs >= optimal_threshold).astype(int)
        dataset[output_col] = preds
        return dataset

    def _adaptive_threshold(self, dataset, column, output_col):
        """آستانه تطبیقی بر اساس ویژگی‌های محلی."""
        adaptive_method = self.config.get('method', 'quantile')

        if adaptive_method == 'quantile':
            window = self.config.get('window', 10)
            quantile = self.config.get('quantile', 0.5)
            probs = dataset[column].values
            preds = np.zeros(len(probs), dtype=int)

            for i in range(len(probs)):
                # پنجره قبلی (یا متقارن؟) - ساده: پنجره گذشته
                start = max(0, i - window + 1)
                window_probs = probs[start:i+1]  # شامل i
                thresh = np.quantile(window_probs, quantile)
                preds[i] = 1 if probs[i] >= thresh else 0

            dataset[output_col] = preds
            return dataset

        elif adaptive_method == 'knn':
            try:
                from sklearn.neighbors import NearestNeighbors
            except ImportError:
                raise ImportError("scikit-learn is required for knn adaptive threshold.")

            # نیاز به داده‌های ویژگی برای یافتن همسایه‌ها
            # فرض می‌کنیم ستون‌های ویژگی در config با نام 'feature_columns' داده شده‌اند
            feature_cols = self.config.get('feature_columns')
            if feature_cols is None:
                # اگر داده نشده، از همه ستون‌های غیر از ستون جاری و output استفاده کن؟
                feature_cols = [c for c in dataset.columns if c != column and c != output_col]
                if not feature_cols:
                    raise ValueError("No feature columns available for knn adaptive threshold.")

            X = dataset[feature_cols].values
            probs = dataset[column].values
            n_neighbors = self.config.get('n_neighbors', 5)
            distance_metric = self.config.get('distance_metric', 'euclidean')

            nn = NearestNeighbors(n_neighbors=n_neighbors, metric=distance_metric)
            nn.fit(X)
            distances, indices = nn.kneighbors(X)

            preds = np.zeros(len(probs), dtype=int)
            for i in range(len(probs)):
                neighbor_probs = probs[indices[i]]
                thresh = np.mean(neighbor_probs)  # یا می‌توان چندک را هم استفاده کرد
                preds[i] = 1 if probs[i] >= thresh else 0

            dataset[output_col] = preds
            return dataset

        else:
            raise ValueError(f"Unsupported adaptive method: {adaptive_method}")